In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 【核心修改】导入 TransformerConv 层
from torch_geometric.nn import TransformerConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

In [2]:
# --- 1. 超参数配置 (新增 heads) ---
hparams = {
    'dataset': 'iris',
    'threshold_pos': 10,
    'threshold_neg': 40000,
    'hidden_channels': 16,
    'heads': 4,  # Transformer的多头注意力头数
    'learning_rate': 0.005,
    'weight_decay': 5e-4,
    'epochs': 100,
    'dropout': 0.5
}

# --- 2. TensorBoard 设置 (与之前相同) ---
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir_name = f"../runs/{hparams['dataset']}_transformer_tp={hparams['threshold_pos']}_tn={hparams['threshold_neg']}_lr={hparams['learning_rate']}_wd={hparams['weight_decay']}_{timestamp}"
writer = SummaryWriter(log_dir_name)
print(f"TensorBoard 日志将保存在: {log_dir_name}")

TensorBoard 日志将保存在: ../runs/iris_transformer_tp=10_tn=40000_lr=0.005_wd=0.0005_20250926-140901


In [3]:
# --- 3. 数据加载与预处理函数 (与之前相同) ---
def load_and_prepare_data(dataset_name, threshold_pos, threshold_neg):
    base_path = f'../data/{dataset_name}/'
    
    features_path = f"{base_path}{dataset_name}.data.cleaned.csv"
    x_numpy = np.loadtxt(features_path, delimiter=',')
    x_features = torch.tensor(x_numpy, dtype=torch.float)
    num_nodes = x_features.shape[0]

    adj_matrix_pos_path = f"{base_path}{dataset_name}_A_plus_UG.csv"
    a_plus_pos_numpy = np.loadtxt(adj_matrix_pos_path, delimiter=',')
    a_plus_pos = torch.tensor(a_plus_pos_numpy, dtype=torch.float)
    a_plus_pos[a_plus_pos <= threshold_pos] = 0
    a_plus_pos.fill_diagonal_(0)
    edge_index_pos, edge_attr_pos = dense_to_sparse(a_plus_pos)

    adj_matrix_neg_path = f"{base_path}{dataset_name}_A_negative_UG.csv"
    a_plus_neg_numpy = np.loadtxt(adj_matrix_neg_path, delimiter=',')
    a_plus_neg = torch.tensor(a_plus_neg_numpy, dtype=torch.float)
    a_plus_neg[a_plus_neg <= threshold_neg] = 0
    a_plus_neg.fill_diagonal_(0)
    edge_index_neg, edge_attr_neg = dense_to_sparse(a_plus_neg)

    labels_path = f"{base_path}{dataset_name}.data"
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
    try:
        df = pd.read_csv(labels_path, header=None, names=column_names)
        labels_numpy = df['species'].values
    except Exception: # For datasets like 'zoo' with different format
        df = pd.read_csv(f"{base_path}{dataset_name}.data.csv")
        labels_numpy = df.iloc[:, -1].values
        
    encoder = LabelEncoder()
    y_numpy = encoder.fit_transform(labels_numpy)
    y = torch.tensor(y_numpy, dtype=torch.long)
    if num_nodes != len(y):
        y = y[:num_nodes]

    data = Data(x=x_features, y=y,
                edge_index_pos=edge_index_pos, edge_attr_pos=edge_attr_pos,
                edge_index_neg=edge_index_neg, edge_attr_neg=edge_attr_neg)

    num_train = int(num_nodes * 0.6)
    num_val = int(num_nodes * 0.2)
    indices = torch.randperm(num_nodes)
    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.train_mask[indices[:num_train]] = True
    data.val_mask[indices[num_train:num_train + num_val]] = True
    data.test_mask[indices[num_train + num_val:]] = True
    
    return data, len(np.unique(y_numpy))

In [4]:
# --- 4. 【核心修改】定义双分支 Graph Transformer 模型 ---
class DualConceptTransformer(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, dropout=0.5):
        super(DualConceptTransformer, self).__init__()
        self.dropout = dropout

        # 分支一：处理正概念格图的 Transformer 层
        self.pos_conv = TransformerConv(in_channels, hidden_channels, heads=heads)
        
        # 分支二：处理负概念格图的 Transformer 层
        self.neg_conv = TransformerConv(in_channels, hidden_channels, heads=heads)
        
        # 融合层
        # 输入维度是两个分支隐藏层维度之和 (hidden_channels * heads * 2)
        self.fusion_layer = nn.Linear(hidden_channels * heads * 2, out_channels)

    def forward(self, x, edge_index_pos, edge_index_neg):
        # --- 分支一前向传播 (正概念图) ---
        # TransformerConv 只需要 x 和 edge_index，它会自己学习边的权重（注意力）
        h_pos = self.pos_conv(x, edge_index_pos)
        h_pos = F.relu(h_pos)
        h_pos = F.dropout(h_pos, p=self.dropout, training=self.training)
        
        # --- 分支二前向传播 (负概念图) ---
        h_neg = self.neg_conv(x, edge_index_neg)
        h_neg = F.relu(h_neg)
        h_neg = F.dropout(h_neg, p=self.dropout, training=self.training)
        
        # --- 特征融合 ---
        h_combined = torch.cat([h_pos, h_neg], dim=1)
        
        # --- 通过融合层得到最终输出 ---
        out = self.fusion_layer(h_combined)
        return out

In [5]:
# --- 5. 实例化数据和模型 (与之前类似) ---
data, num_classes = load_and_prepare_data(hparams['dataset'], hparams['threshold_pos'], hparams['threshold_neg'])

model = DualConceptTransformer(in_channels=data.num_node_features, 
                               hidden_channels=hparams['hidden_channels'], 
                               out_channels=num_classes,
                               heads=hparams['heads'],
                               dropout=hparams['dropout'])

optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'], weight_decay=hparams['weight_decay'])
criterion = torch.nn.CrossEntropyLoss()

In [6]:
# --- 6. 训练与评估函数 (模型调用部分有修改) ---
def train(epoch):
    model.train()
    optimizer.zero_grad()
    # 【修改】模型调用不再需要 edge_attr
    out = model(data.x, data.edge_index_pos, data.edge_index_neg)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    writer.add_scalar('Loss/train', loss.item(), epoch)
    return loss.item()

def evaluate(epoch):
    model.eval()
    with torch.no_grad():
        # 【修改】模型调用不再需要 edge_attr
        out = model(data.x, data.edge_index_pos, data.edge_index_neg)
        pred = out.argmax(dim=1)
        
        train_acc = (pred[data.train_mask] == data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
        val_acc = (pred[data.val_mask] == data.y[data.val_mask]).sum().item() / data.val_mask.sum().item()
        test_acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()

        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/validation', val_acc, epoch)
        writer.add_scalar('Accuracy/test', test_acc, epoch)
        
        return train_acc, val_acc, test_acc

In [7]:
# --- 7. 主训练循环 (与之前相同) ---
print("\n--- 开始训练 (双概念格 Graph Transformer) ---")
for epoch in range(1, hparams['epochs'] + 1):
    loss = train(epoch)
    if epoch % 1 == 0:
        train_acc, val_acc, test_acc = evaluate(epoch)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

# --- 训练完成后 ---
final_train_acc, final_val_acc, final_test_acc = evaluate(hparams['epochs'])
print(f'--- 训练完成 ---')
print(f'最终测试集准确率 (双概念格分支 Transformer): {final_test_acc:.4f}')

metrics = {
    'accuracy/final_train': final_train_acc,
    'accuracy/final_validation': final_val_acc,
    'accuracy/final_test': final_test_acc
}

writer.add_hparams(hparams, metrics)
writer.close()


--- 开始训练 (双概念格 Graph Transformer) ---
Epoch: 001, Loss: 1.0886, Train Acc: 0.7889, Val Acc: 0.8667, Test Acc: 0.7667
Epoch: 002, Loss: 1.0083, Train Acc: 1.0000, Val Acc: 0.9667, Test Acc: 1.0000
Epoch: 003, Loss: 0.9236, Train Acc: 1.0000, Val Acc: 0.9667, Test Acc: 1.0000
Epoch: 004, Loss: 0.8293, Train Acc: 1.0000, Val Acc: 0.9667, Test Acc: 1.0000
Epoch: 005, Loss: 0.7472, Train Acc: 1.0000, Val Acc: 0.9667, Test Acc: 1.0000
Epoch: 006, Loss: 0.6487, Train Acc: 1.0000, Val Acc: 0.9667, Test Acc: 1.0000
Epoch: 007, Loss: 0.5477, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 008, Loss: 0.4400, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 009, Loss: 0.3571, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 010, Loss: 0.2916, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 011, Loss: 0.1998, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 012, Loss: 0.1743, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 013, L